### Dependencies

In [1]:
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from sqlalchemy import update

# ETL Project

## Import Database 2 - World Development Indicators

### Conect to Database

In [2]:
rds_connection_string = "postgres:@Pifarus_1190@localhost:5432/ELT_Happiness"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
# Confirm tables
engine.table_names()

['indicators_data', 'hapiness', 'country', 'indicators']

In [4]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)
Hapiness = Base.classes.hapiness
Country = Base.classes.country
Indicators = Base.classes.indicators

In [5]:
session = Session(engine)

In [6]:
country_code = session.query(Country.country_code).\
                filter(Country.country_name == Hapiness.country_name).all()

In [7]:
len(country_code)

121

### Integrity between Hapiness vs Country Code

In [8]:
country_query = session.query(Hapiness.country_name, Country.country_code).\
                    join(Country, Hapiness.country_name == Country.country_name, isouter=True).\
                    filter(Country.country_code==None).all()

In [9]:
country_query

[('Tanzania', None),
 ('Syria', None),
 ('Russia', None),
 ('United States', None),
 ('Ivory Coast', None),
 ('Vietnam', None),
 ('Congo (Brazzaville)', None),
 ('Netherlands', None),
 ('Central African Republic', None),
 ('Niger', None),
 ('South Korea', None),
 ('Philippines', None),
 ('Czech Republic', None),
 ('Iran', None),
 ('Palestinian Territories', None),
 ('United Arab Emirates', None),
 ('Venezuela', None),
 ('United Kingdom', None),
 ('Kosovo', None),
 ('Congo (Kinshasa)', None),
 ('Dominican Republic', None),
 ('Moldova', None),
 ('Bolivia', None)]

In [10]:
for item in country_query:
    countryname = item[0].replace(" ", "").lower()
    toupdate=item[0]

    try:
        country_query2 =session.query(Country.country_name).\
            filter(Country.new_name.like("%"+countryname+"%")).all()
        
        if len(country_query2)>1:
            print(country_query2)
            numif=int(input(f'1 to {len(country_query2)}:'))
            num = numif-1
        else:
            num=0
        session.query(Country).filter(Country.country_name==country_query2[num][0]).\
            update({'country_name':toupdate},synchronize_session=False)
    except:
        print(f'error:{countryname}')
        pass

[('United States Minor Outlying Islands (the)',), ('United States of America (the)',)]
1 to 2:2
error:ivorycoast
error:congo(brazzaville)
[('Niger (the)',), ('Nigeria',)]
1 to 2:1
error:southkorea
error:czechrepublic
error:palestinianterritories
error:kosovo
error:congo(kinshasa)


In [11]:
country_query3 = session.query(Hapiness.country_name, Country.country_code).\
                    join(Country, Hapiness.country_name == Country.country_name, isouter=True).\
                    filter(Country.country_code==None).all()
country_query3

[('Ivory Coast', None),
 ('Congo (Brazzaville)', None),
 ('South Korea', None),
 ('Czech Republic', None),
 ('Palestinian Territories', None),
 ('Kosovo', None),
 ('Congo (Kinshasa)', None)]

#### By looking for database

In [12]:
#update Côte d'Ivoire to Ivory Cost
session.query(Country).filter(Country.country_name=="Côte d'Ivoire").\
            update({'country_name':"Ivory Coast"},synchronize_session=False)

1

In [13]:
#update Korea (the Republic of) to South Korea
session.query(Country).filter(Country.country_name=="Korea (the Republic of)").\
            update({'country_name':"South Korea"},synchronize_session=False)

1

In [14]:
#update Czechia to Czech Republic
session.query(Country).filter(Country.country_name=="Czechia").\
            update({'country_name':"Czech Republic"},synchronize_session=False)

1

In [15]:
#update Czechia to Palestinian Territories
session.query(Country).filter(Country.country_name=="Palestine, State of").\
            update({'country_name':"Palestinian Territories"},synchronize_session=False)

1

In [16]:
#update Congo (the) to Congo (Brazzaville)
session.query(Country).filter(Country.country_name=="Congo (the)").\
            update({'country_name':"Congo (Brazzaville)"},synchronize_session=False)

1

In [17]:
#update Congo (the Democratic Republic of the) to Congo (Kinshasa)
session.query(Country).filter(Country.country_name=="Congo (the Democratic Republic of the)").\
            update({'country_name':"Congo (Kinshasa)"},synchronize_session=False)

1

In [18]:
country_query4 = session.query(Hapiness.country_name, Country.country_code).\
                    join(Country, Hapiness.country_name == Country.country_name, isouter=True).\
                    filter(Country.country_code==None).all()
country_query4

[('Kosovo', None)]

### Selecting Countries

In [19]:
country_code = session.query(Country.country_code).\
                filter(Country.country_name == Hapiness.country_name).all()

In [20]:
number_countries = len(country_code)

### Review Indicators Data

In [21]:
#Import Information

indicators_df = pd.read_csv('indicators.csv')
indicators_df.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


In [22]:
#Evaluate Duplicates
indicators_df = indicators_df.drop_duplicates()

#### Create Indicators Table

In [23]:
data_indicators_df=indicators_df[['IndicatorName','IndicatorCode']].drop_duplicates()
data_indicators_df=data_indicators_df.rename(columns={'IndicatorCode':'indicator_code',
                                                     'IndicatorName':'indicator_name'})

data_indicators_df.head()

,indicator_name,indicator_code
0,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT
1,Age dependency ratio (% of working-age populat...,SP.POP.DPND
2,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL
3,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG
4,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD


#### Create Data Table with relevant information

In [24]:
info = indicators_df[['CountryCode','IndicatorCode','Year','Value']]
data =[]
for country in country_code:
    country = country[0]
    data.append(info.loc[indicators_df["CountryCode"]==country])

In [25]:
data_df = pd.concat(data)
len(data_df)

3807665

In [26]:
data_df = data_df.loc[data_df['Year']>=2000]
len(data_df)

1676111

In [27]:
number_countries = len(data_df['CountryCode'].unique())
number_countries

140

In [28]:
#Evaluate consistency of data across Indicators/Years/Countries
#how many countries has reported which indicators by year
data_consistency = pd.DataFrame(data_df.groupby(['IndicatorCode','Year'])['CountryCode'].count())

In [29]:
#Evaluate indicators reported consistently in all data countri
data = data_consistency.loc[data_consistency['CountryCode']==number_countries]

In [30]:
data = data.reset_index()
indicator = data['IndicatorCode'].unique()
years = data['Year'].unique()

In [31]:
data =[]
for ind in indicator:
    for year in years:
        dataview = data_df.loc[data_df['Year']==year]
        data.append(dataview.loc[dataview["IndicatorCode"]==ind])

In [32]:
data_df = pd.concat(data)
len(data_df)

260074

In [33]:
data_df = data_df.rename(columns={'CountryCode':'country_code',
                                 'IndicatorCode':'indicator_code',
                                 'Year':'year',
                                 'Value':'value'})

### Connect to Database

In [34]:
rds_connection_string = "postgres:@Pifarus_1190@localhost:5432/ELT_Happiness"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Load to Database

In [35]:
data_df.to_sql(name='indicators_data', con=engine, if_exists='append', index=False)

In [36]:
data_indicators_df.to_sql(name='indicators', con=engine, if_exists='append', index=False)